In [1]:
import pandas as pd
import json
import ast

In [2]:
pd.set_option('display.max_colwidth', None) ##para ver todo el contenido de la columna

In [3]:
# traigo el dataset credits a un dataframe y le cambio el nombre de la columna del id(pelicula) a id_original
dfcredits= pd.read_csv(r'C:\Users\flori\Desktop\DATA SCIENCE\LABS\credits\credits.csv')
dfcredits.rename(columns={'id':'id_original'}, inplace= True)


In [4]:
#utilizo ast literal para evaluar y transformar las cadenas de texto que representan diccionarios a diccionarios propiamente dichos
dfcredits['crew'] = dfcredits['crew'].apply(ast.literal_eval) 

In [6]:
df_exploded = dfcredits.explode('crew')

def extract_crew_info(x):
    if isinstance(x, dict):
        return pd.Series([x.get('credit_id', ''), x.get('department', ''),x.get('gender', ''),x.get('id', ''),x.get('job', ''),x.get('name', ''),x.get('profile_path', '')  ])
    else:
        return pd.Series(['', ''])

df_exploded[['credit_id', 'department', 'gender','id_crew','job', 'name', 'profile_path']] = df_exploded['crew'].apply(extract_crew_info) 

In [22]:
df = dfcredits.join(df_exploded[['credit_id', 'department', 'gender', 'id_crew', 'job', 'name', 'profile_path']]) 

In [23]:
columna_eliminada = ['crew','cast', 'credit_id','profile_path', 'gender', 'id_crew', 'department']
df.drop(columns=columna_eliminada, inplace=True) 




In [24]:
df.head()

,id_original,job,name
0,862,Director,John Lasseter
0,862,Screenplay,Joss Whedon
0,862,Screenplay,Andrew Stanton
0,862,Screenplay,Joel Cohen
0,862,Screenplay,Alec Sokolow


In [25]:
# Convertir la columna numéricas a cadenas

df['id_original'] = df['id_original'].astype(str) 

In [26]:
director_resultado = df[df['job'] == 'Director']['job'].value_counts()
director_resultado 

Director    49048
Name: job, dtype: int64

In [27]:
df_director= df[df['job']== 'Director']
df_director.head()

,id_original,job,name
0,862,Director,John Lasseter
1,8844,Director,Joe Johnston
2,15602,Director,Howard Deutch
3,31357,Director,Forest Whitaker
4,11862,Director,Charles Shyer


In [28]:
df_director.shape

(49048, 3)

In [18]:
grouped_df= df_director[['id_original', 'name']].groupby('name').count().reset_index()

In [19]:
grouped_df.to_csv('directoresPorpeliID.csv', index=False)

In [21]:
dfori= grouped_df[grouped_df['id_original']=='826']
dfori

,name,id_original


In [ ]:
# identificar Director por cada Id Pelicula

id_director = df_director[df_director['id_original'] == '862']
director_names = id_director['name'].values[0].split(',')
director_name = director_names[0]

print(f"ID: 862, Director: {director_name}")


ID: 862, Director: John Lasseter


In [ ]:
dfcredits1= dfcredits['crew']